#  My business plan is to utilize the Foursquare API to find the top rated restaurant of multiple types in each cluster. This will enable people to find Italian restaurants within one tenth of the city. 

# My data needs will be utilizing ratings, location data, and clustering from postal codes to make the maps. This will require sorting into clusters of the city, finding centers of said clusters and locating Italian restaurants close to that.


In [1]:
#importing necessary programs
import pandas as pd
import numpy as np
from numpy import nan
import sklearn
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#importing neighborhoods and postal data from Wikipedia into dataframe
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)
neighborhoods = tables[0]
neighborhoods.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [5]:
#replacing not assigned values with nan to remove
neighborhoods['Borough'] = neighborhoods['Borough'].replace(['Not assigned'],nan)
neighborhoods['Neighbourhood'] = neighborhoods['Neighbourhood'].replace(['Not assigned'],nan)
neighborhoods.dropna(how='any', inplace=True)
neighborhoods.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [6]:
neighborhoods.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

In [8]:
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
#double checking there are no duplicates. According to Wikipedia there are 103 postalcodes
neighborhoods.shape


(103, 3)

In [12]:
#Assigning coordinates to postalcodes from previous csv file
coordinates = pd.read_csv('Geospatial_coordinates.csv')

In [14]:
coordinates.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
neighborhoods = pd.merge(neighborhoods, coordinates, left_on=['PostalCode'], right_on=['PostalCode'])
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [16]:
#Clustering neighborhoods into 10 clusters
k=10 
coordinates.set_index('PostalCode', inplace=True)
clustering = KMeans(n_clusters=k, random_state=0).fit(coordinates)
coordinates.insert(0,'Cluster', clustering.labels_)
neighborhoods = pd.merge(neighborhoods, coordinates, left_on=['PostalCode'], right_on=['PostalCode'])
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude_x,Longitude_x,Cluster,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,9,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,9,43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,8,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,7,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188,6,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,9,43.657162,-79.378937


In [18]:
#dropping repeated columns
neighborhoods.drop('Latitude_y', axis=1, inplace=True)
neighborhoods.drop('Longitude_y', axis=1, inplace=True)
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude_x,Longitude_x,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,2
1,M4A,North York,Victoria Village,43.725882,-79.315572,1
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,9
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,9
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,8
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,7
7,M3B,North York,Don Mills,43.745906,-79.352188,6
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,9


In [20]:
neighborhoods.rename(columns={'Latitude_x':'Latitude', 'Longitude_x':'Longitude'}, inplace=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,2
1,M4A,North York,Victoria Village,43.725882,-79.315572,1
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,9
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,9


In [22]:
#creating folium base map of toronto
import folium
toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)
toronto

In [23]:
#plotting points on map
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['PostalCode'], neighborhoods['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto)
       
toronto

In [28]:
centers = clustering.cluster_centers_

In [40]:
#Calling API
import geopy
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
#split Centers list into latitude list and longitude list
latitude = []
longitude = []
for i in centers:
   latitude.append(i[0])
   longitude.append(i[1])
print(latitude)
print(longitude)

[43.71704775714286, 43.69170681666667, 43.784528125, 43.738124285714285, 43.67220162727273, 43.7318991, 43.74926703846154, 43.79238216, 43.63623246666667, 43.661070808333335]
[-79.55512174285715, -79.32991851666667, -79.3131013, -79.49007181428571, -79.45350943636363, -79.25759814285715, -79.40275245384615, -79.1932231, -79.53902705555555, -79.3867346125]


In [70]:
#Getting Italian results within 3000 feet of each center
CLIENT_ID = 'ZAJLYQRVEZVEGOO5OR3FA1HJDPZG0K0NJMSMD2YCI0TMLXKW' # your Foursquare ID
CLIENT_SECRET = 'VRRA2TRBABSTJULUVWSMH4431J452FXQ54T5I4QRGHJ4M2MN' # your Foursquare Secret
VERSION = '20201101'
LIMIT = 30
search_query = 'Italian'
radius = 3000
lat = latitude[i]
long = longitude[i]
i = 0
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)

In [95]:
#creating Dataframes for each cluster of the city
import requests
i = 0
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster0 = json_normalize(venues)
cluster0.drop(['categories', 'referralId','hasPerk','location.address', 'location.crossStreet', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster0.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster0.head()

,id,name,latitude,longitude,address
0,4b199b46f964a5205be023e3,Jolly II Italian Restaurant,43.711946,-79.53151,"[2625 Weston (401), Toronto ON, Canada]"


In [98]:
i = 1
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster1 = json_normalize(venues)
cluster1.drop(['categories', 'referralId','hasPerk','location.address', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster1.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster1.head()

,id,name,latitude,longitude,address
0,4c2b9fe577cfe21e4cccb4f1,North Pole Italian Bakery,43.681517,-79.331810,"[1210 Danforth Ave., Toronto ON, Canada]"
1,4de024f0b0fbe2cfa5fee3c4,Florentina's Italian Restaurant,43.676562,-79.355699,"[Toronto ON, Canada]"
2,51a37366498e30597bfce92d,Tratoria Italiano,43.693513,-79.349925,[Canada]


In [100]:
i = 2
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster2 = json_normalize(venues)
cluster2.drop(['categories', 'referralId','hasPerk','location.address', 'location.crossStreet', 'location.labeledLatLngs', 'location.distance', 'location.postalCode', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster2.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster2.head()

,id,name,latitude,longitude,address
0,4b5a2bf2f964a520deb128e3,Remezzo Italian Bistro,43.778649,-79.308264,"[3335 Sheppard Ave E (at Warden Ave.), Scarbor..."
1,4b6755d0f964a520c6482be3,Basilico Italian Bistro,43.786527,-79.328686,"[2746 Victoria Park Ave., Toronto ON, Canada]"


In [102]:
i = 3
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster3 = json_normalize(venues)
cluster3.drop(['categories', 'referralId','hasPerk','location.address', 'location.postalCode', 'location.crossStreet', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster3.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster3.head()

,id,name,latitude,longitude,address
0,4de3e16efa7651589f21395e,Saggio Italian Eatery & Espresdo Bar,43.758340,-79.476741,"[Toronto ON, Canada]"
1,52b5da0e498e96708cf1c974,Cumpari's Italian Eatery,43.732118,-79.459921,"[3610 dufferin street (Dufferin/Wilson), Toron..."
2,4d8b75a17d4c548173907e71,Commisso Bros & Racco Italian Bakery,43.748606,-79.517274,"[33 Eddystone, Toronto ON, Canada]"
3,4cfe726a084f54811f969009,Dora's Italian,43.768120,-79.475147,"[Toronto ON, Canada]"


In [120]:
i = 4
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster4 = json_normalize(venues)
cluster4.drop(['categories','location.postalCode','referralId','hasPerk','location.address', 'location.crossStreet', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster4.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster4.head(30)

,id,name,latitude,longitude,address,location.neighborhood
0,4aedafd1f964a52025ce21e3,Commisso Bros. & Racco Italian Bakery,43.697621,-79.468550,"[8 Kincort Street (Castlefield and Kincort), T...",NaN
1,4e8218e1775b313c2afa5d8d,Agio Italian Ristorante,43.676156,-79.450574,"[1351 St Clair West (Lansdowne), Toronto ON, C...",NaN
2,4e31afdd091a973ec9c5a2b5,"Punto Gelato, Simply Italian",43.669955,-79.392603,"[146 Cumberland St (btwn Avenue Rd & Bay St), ...",Yorkville
3,52f6816f11d24a43115dc834,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...",NaN
4,546e313c498ea838219b4ce7,The Italian Sandwich,43.680377,-79.427523,"[242 Rushton Road, Toronto ON, Canada]",NaN
5,521ce6b811d235da80b670d2,Italian Canadian Savings & Credit Union,43.649369,-79.421831,"[1168 Dundas Street West, Toronto ON M6j 1x4, ...",NaN
6,4fe3bf67e4b0ef61f4cbe022,Italian Walk Of Fame,43.655182,-79.415781,"[Toronto ON, Canada]",NaN
7,4cfc22012d80a14304854fd8,Catania Italian Cuisine,43.680797,-79.430570,"[Toronto ON, Canada]",NaN
8,4fdc81d7e4b090d3f5a66bf7,Italian Chamber Of Commerce,43.656158,-79.409106,[Canada],NaN
9,52430d7611d2b3a076a88132,Italian canadian savings and credit union,43.708357,-79.453877,"[Dufferin, Toronto ON, Canada]",NaN


In [110]:
i = 5
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster5 = json_normalize(venues)
cluster5.drop(['categories','referralId','hasPerk', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.country'], axis=1, inplace=True)
cluster5.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster5.head()

,id,name,latitude,longitude,address
0,5207972e11d28a4dc2aef15a,Italian Espresso Bar,43.715701,-79.2818,[Canada]


In [111]:
i = 6
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster6 = json_normalize(venues)
cluster6.drop(['categories','location.postalCode','referralId','hasPerk','location.address', 'location.crossStreet', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster6.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster6.head()

,id,name,latitude,longitude,address
0,4b9fbb53f964a5206a3937e3,Italian Cafe,43.723770,-79.397573,"[2200 Yonge St, Toronto ON, Canada]"
1,519e5c39498eb25c945e98c8,VIA CIBO | italian streetfood,43.754067,-79.357951,"[808 York Mills Rd, Unit 14 (Leslie St), Toron..."
2,4b69ad98f964a520f3ac2be3,Paisano's,43.764518,-79.399898,"[116 Willowdale Ave (Don Mills), Toronto ON M3..."


In [114]:
i = 7
lat = latitude[i]
long = longitude[i]
radius = 5000
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster7 = json_normalize(venues)
cluster7.drop(['categories','location.postalCode','referralId','hasPerk','location.address', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster7.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster7.head()

,id,name,latitude,longitude,address
0,5785be37498e9bd8a28941b0,Scaddabush Italian Kitchen & Bar,43.777460,-79.254358,"[580 Progress Ave., Scarborough ON M1P 2K2, Ca..."
1,4b107754f964a520147123e3,Joey Bravo's Italian Restaurant,43.788071,-79.265134,"[4505 Sheppard Ave E, Scarborough ON M1S 1V3, ..."


In [116]:
i = 8
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster8 = json_normalize(venues)
cluster8.drop(['categories','location.postalCode','referralId','hasPerk','location.address', 'location.crossStreet', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster8.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster8.head(10)

,id,name,latitude,longitude,address,venuePage.id
0,4e8e465e722e5135d891fabb,Guelph Italian Canadian Club,43.628227,-79.485596,[Canada],NaN
1,50884667e4b0835853331519,Molinaro's fine italian foods pizza,43.600146,-79.580803,"[2345 Stanfield Rd., Unit 5, Mississauga ON L4...",58401121
2,5a492661c4df1d5eb27d8324,Scaddabush Italian Kitchen & Bar,43.614226,-79.559943,"[1900 The Queensway, Etobicoke ON M9C 5H5, Can...",NaN
3,5d017276a8eb60002cc45b82,Scaddabush Italian Kitchen & Bar,43.622800,-79.479029,"[122 Marine Parade Dr, Toronto ON M8V 0E7, Can...",NaN
4,4b97dd5df964a520091b35e3,Cordi Italian Bakery,43.636364,-79.596070,"[1891 Rathburn Rd E (at Ponytrail Dr.), Missis...",NaN
5,52601343498efa88ffcfc7c8,Istituto Italiano di Cultura,43.596151,-79.515397,"[496 Huron Street, Toronto ON, Canada]",NaN


In [125]:
i = 9
lat = latitude[i]
long = longitude[i]
url ='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
cluster9 = json_normalize(venues)
cluster9.drop(['categories','location.postalCode','referralId','hasPerk','location.address', 'location.crossStreet', 'location.labeledLatLngs', 'location.distance', 'location.distance', 'location.cc', 'location.city', 'location.country', 'location.state'], axis=1, inplace=True)
cluster9.rename(columns={'location.lat':'latitude', 'location.lng':'longitude', 'location.formattedAddress':'address'}, inplace=True)
cluster9.head(40)

,id,name,latitude,longitude,address,location.neighborhood,venuePage.id
0,52f6816f11d24a43115dc834,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...",NaN,NaN
1,4e31afdd091a973ec9c5a2b5,"Punto Gelato, Simply Italian",43.669955,-79.392603,"[146 Cumberland St (btwn Avenue Rd & Bay St), ...",Yorkville,NaN
2,5b897e92db1d81002c91df8c,Fabbrica Rustic Italian,43.647161,-79.381691,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]",NaN,NaN
3,5e594c8a3de308000870c948,Elm Street Italian Deli,43.657690,-79.382480,"[15 Elm Street, Toronto ON M5G 1G7, Canada]",NaN,NaN
4,4bfc0289c3ba9521c00f9653,Italian Consulate Toronto,43.654027,-79.394104,"[136 Beverley St (Dundas Street), Toronto ON, ...",NaN,NaN
5,573df789498e03dd8e54b166,Mustachio Italian Eatery,43.656160,-79.383190,"[595 Bay St (Dundas St), Toronto ON M5G 2C2, C...",NaN,NaN
6,51bf3866498e55ee55df8db0,The Fresh Italian,43.654991,-79.387897,"[Toronto ON, Canada]",NaN,NaN
7,53daae5b498e9c9597c19b23,john's italian cafe,43.656127,-79.393301,"[27 Baldwin Street, Toronto ON, Canada]",NaN,NaN
8,526fe29411d2aeb3803013b0,The Fresh Italian Eatery,43.653889,-79.390785,"[109 McCaul Street, Unit #42 (Dundas Street We...",NaN,NaN
9,53bed05e498e908d95ce1f4d,italian vegatarian magic,43.673801,-79.386610,"[7 Yorkville Ave., Toronto ON, Canada]",NaN,NaN


These are the italian restaurants in each of the 10 clusters of the city. Clearly if one wanted to eat at an Italian restaurant, 4 and 9 seem like the best clusters to visit. 